In [1]:
import pandas as pd
import urllib

import numpy as np

import json

from tqdm.autonotebook import tqdm

#%matplotlib inline

tqdm.pandas()

import dask.dataframe as dd

from dask.multiprocessing import get
from dask.diagnostics import ProgressBar

from datetime import datetime
import matplotlib.pyplot as plt

from IPython.display import display


<ipython-input-1-5cfb607d7a2e>:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import urllib3

In [3]:
http = urllib3.PoolManager()

In [4]:
from config_batch import * 

# Functions

In [5]:
ws_hostname = "127.0.1.1"
ws_hostname = "10.1.0.45"

# ws_hostname = "192.168.1.3"


In [6]:
def call_ws(addr_data, check_result=True, structured_osm=False): #lg = "en,fr,nl"
    t = datetime.now()
    
    params = urllib.parse.urlencode({"street": addr_data[street_field],
                                     "housenumber": addr_data[housenbr_field],
                                     "city": addr_data[city_field],
                                     "postcode": addr_data[postcode_field],
                                     "country": addr_data[country_field],
                                     "check_result" : "yes" if check_result else "no",
                                     "struct_osm" : "yes" if structured_osm else "no"
                                    })
    url = f"http://{ws_hostname}:5000/search/?{params}"
    
    try:
        with urllib.request.urlopen(url) as response:
            res = response.read()
            res = json.loads(res)
#             print(res)
            res["time"] = datetime.now() - t
            return res
    except Exception as e:
        return str(e)
    

In [7]:
def call_ws_batch(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False): #lg = "en,fr,nl"
#     print(addr_data)
#     print(addr_data.shape)
#     print()
    file_data = addr_data.rename(columns = {
        street_field : "street",
        housenbr_field: "housenumber",
        postcode_field: "postcode",
        city_field: "city",
        country_field: "country",
        addr_key_field : "addr_key"
    }).to_csv(index=False)
    
    r = http.request(
    'POST',
    f'http://{ws_hostname}:5000/batch',
    fields= { 
        'media': ('addresses.csv', file_data),
        'mode': mode,
        "with_rejected" : "yes" if with_reject else "no",
        "check_result" : "yes" if check_result else "no",
        "struct_osm" : "yes" if structured_osm else "no"
    })
    
    try:
        res = pd.DataFrame(json.loads(r.data.decode('utf-8')))
    except ValueError:
        print("Cannot decode result:")
        print(json.loads(r.data.decode('utf-8')))
        return 
#     display(res)
    return res

In [8]:
def expand_json(addresses):
    addresses["status"]= addresses.json.apply(lambda d: "error" if "error" in d else "match" if "match" in d else "rejected")
    addresses["time"]  = addresses.json.apply(lambda d: d["time"])

    addresses["timing"]  = addresses.json.apply(lambda d: d["timing"] if "timing" in d else {})

    addresses["method"]= addresses.json.apply(lambda d: d["match"][0]["method"] if len(d)>0 and "match" in d else "none")
    
    for field in ["street", "number", "postcode", "city"]:
        addresses[field]= addresses.json.apply(lambda d: d["match"][0]["addr_out_"+field] if len(d)>0 and "match" in d else "")
    return 

# Calls

## Single address calls

In [12]:
call_ws({street_field:   "Av. Fonsny", 
         housenbr_field: "20",
         city_field:     "Saint-Gilles",
         postcode_field: "1060",
         country_field:  "Belgium"}, check_result=True, structured_osm=False)

{'match': [{'SIM_city': 0.46153846153846156,
   'SIM_house_nbr': 0.0,
   'SIM_street': 1.0,
   'SIM_street_which': 'addr_out_street',
   'SIM_zip': 1.0,
   'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '',
   'addr_out_postcode': '1060',
   'addr_out_street': '',
   'display_name': 'Saint-Gilles - Sint-Gillis, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1060, België / Belgique / Belgien',
   'extra_house_nbr': '20',
   'lat': '50.829366783257896',
   'lon': '4.344772645328859',
   'method': 'nostreet',
   'osm_addr_in': '1060 Saint-Gilles, Belgium',
   'place_id': 123361,
   'place_rank': 21}],
 'rejected': [],
 'time': datetime.timedelta(seconds=1, microseconds=736013)}

In [13]:
call_ws({street_field:   "Avenue Louise", 
         housenbr_field: "375",
         city_field:     "Ixelles",
         postcode_field: "1050",
         country_field:  "Belgium"}, check_result=True, structured_osm=True)

{'rejected': [], 'time': datetime.timedelta(seconds=9, microseconds=924150)}

In [17]:
call_ws({street_field:   "Fechtergasse 16/13", 
         housenbr_field: "",
         city_field:     "Wenen",
         postcode_field: "1090",
         country_field:  "Oostenrijk"}, check_result=False, structured_osm=False)

{'match': [{'SIM_street_which': '',
   'addr_out_city': 'Alsergrund',
   'addr_out_country': 'Österreich',
   'addr_out_number': '',
   'addr_out_postcode': '1090',
   'addr_out_street': '',
   'display_name': 'KG Alsergrund, Alsergrund, Wien, 1090, Österreich',
   'extra_house_nbr': '16/13',
   'lat': '48.22281431273457',
   'lon': '16.3564883123584',
   'method': 'nostreet',
   'osm_addr_in': '1090 Wenen, Oostenrijk',
   'place_id': 504963,
   'place_rank': 21}],
 'rejected': [],
 'time': datetime.timedelta(seconds=1, microseconds=421142)}

In [22]:
call_ws({street_field:   "Fechtergasse 16/13 1090 Wenen", 
         housenbr_field: "",
         city_field:     "",
         postcode_field: "",
         country_field:  "Oostenrijk"}, check_result=False, structured_osm=False)

{'match': [{'SIM_street_which': '',
   'addr_out_city': 'Alsergrund',
   'addr_out_country': 'Österreich',
   'addr_out_number': '',
   'addr_out_postcode': '1090',
   'addr_out_street': '',
   'display_name': 'KG Alsergrund, Alsergrund, Wien, 1090, Österreich',
   'extra_house_nbr': '16/13',
   'lat': '48.22281431273457',
   'lon': '16.3564883123584',
   'method': 'libpostal+nostreet',
   'osm_addr_in': '1090 wenen, oostenrijk',
   'place_id': 504963,
   'place_rank': 21}],
 'rejected': [],
 'time': datetime.timedelta(seconds=5, microseconds=226816)}

## Batch calls (row by row)

In [126]:
addresses = get_addresses("address.csv.gz")
addresses = addresses.sample(100).copy()

### Simple way

In [74]:
addresses["json"] = addresses.progress_apply(call_ws, check_result=True, structured_osm=False, axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

### Using Dask

In [17]:
dd_addresses = dd.from_pandas(addresses, npartitions=4)

dask_task = dd_addresses.apply(call_ws, meta=('x', 'str'), axis=1)

with ProgressBar(): 
    addresses["json"] = dask_task.compute()

[########################################] | 100% Completed | 59.1s


In [26]:
expand_json(addresses)

In [27]:
addresses

,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber,json,status,time,timing,method,street,number,postcode,city
1321156,0836.380.520,Belgique,7911,Frasnes-lez-Anvaing,Rue Goderneau(MB),14,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.960761,{},regex[init],Rue Goderneau,,7906,Ath
2660061,2.231.595.460,Belgique,9981,Sint-Laureins,Vlamingstraat(STM),92,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:01.143796,{},regex[init],Vlamingstraat,,9981,Sint-Laureins
164680,0431.356.723,Belgique,1050,Ixelles,Rue de l'Amazone,21,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.724923,{},orig,Rue de l'Amazone - Amazonestraat,21,1050,Ixelles - Elsene
1295595,0832.380.556,Belgique,4000,Liège,Rue Henri-Maus,29,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.584219,{},orig,Rue Henri Maus,,4000,Liège
2293307,2.168.200.715,Belgique,8300,Knokke-Heist,Zeedijk-Knokke,641,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.548851,{},orig,Zeedijk Knokke,,8300,Knokke-Heist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2751112,2.244.614.642,Belgique,9240,Zele,Smeiersberg,5,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.544249,{},orig,Smeiersberg,,9240,Zele
2810314,2.252.606.947,Belgique,1080,Molenbeek-Saint-Jean,Rue Henri De Saegher,27,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.968186,{},orig,Rue Henri De Saegher - Henri De Saegherstraat,27,1080,Molenbeek-Saint-Jean - Sint-Jans-Molenbeek
732484,0650.498.331,Belgique,4700,Eupen,Stockbergerweg,5,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.987093,{},orig,Stockbergerweg,,4700,Eupen
1799145,2.037.557.947,Belgique,1180,Uccle,Parvis Saint-Pierre,119,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0 days 00:00:00.517703,{},orig,Parvis Saint-Pierre - Sint-Pietersvoorplein,,1180,Uccle - Ukkel


## Batch calls (batch WS)

### Single block

In [53]:
# Only geocoding
call_ws_batch(addresses, mode="geo", check_result=True)

,addr_key,lat,lon,place_rank,method
0,0308.251.647,50.467047,4.207648,26.0,orig
1,0408.704.352,50.894373,4.379952,30.0,orig
2,0413.358.372,51.220346,4.413114,30.0,orig
3,0431.356.723,50.824780,4.359237,30.0,orig
4,0439.768.306,50.190684,4.535742,26.0,orig
...,...,...,...,...,...
93,0453.040.676,50.270696,5.521461,26.0,nonum
94,0535.902.531,50.292862,5.031069,30.0,libpostal+regex[lpost]+photon
95,2.202.472.003,50.878578,4.409547,26.0,libpostal+regex[lpost]+photon
96,2.237.979.050,50.590356,5.747109,26.0,libpostal+regex[lpost]+photon


In [62]:
# Geocode + address
call_ws_batch(addresses, mode="short") 

,addr_key,lat,lon,place_rank,method,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country
0,2.164.397.226,50.712641,4.530106,26.0,orig,Rue Robert Boisacq,None,9 A,1330,Rixensart,België / Belgique / Belgien
1,2.279.599.374,50.859702,2.828440,26.0,orig,Stijn Streuvelswijk,None,18,8908,Ieper,België / Belgique / Belgien
2,0669.833.203,51.209175,4.470072,30.0,orig,Boterlaarbaan,239,239,2100,Deurne,België / Belgique / Belgien
3,2.243.108.073,50.658359,5.483832,26.0,orig,Rue de l'Aéropostale,None,8,4460,Grâce-Hollogne,België / Belgique / Belgien
4,2.045.477.107,51.100407,3.167726,26.0,orig,Koningin Astridstraat,None,109,8210,Zedelgem,België / Belgique / Belgien
...,...,...,...,...,...,...,...,...,...,...,...
92,0857.952.627,50.834163,3.231044,26.0,regex[init],Heulsestraat,None,3,8501,Kortrijk,België / Belgique / Belgien
93,2.249.583.121,51.194730,4.407469,30.0,orig,Desguinlei,88-90,90,2018,Antwerpen,België / Belgique / Belgien
94,0834.315.707,51.204519,4.398727,30.0,orig,Diercxsensstraat,39,39,2018,Antwerpen,België / Belgique / Belgien
95,0857.635.002,50.843955,4.264415,26.0,orig,Ninoofsesteenweg,None,227,1700,Dilbeek,België / Belgique / Belgien


In [63]:
# Geocode + address, with rejected addresses
call_ws_batch(addresses, mode="long", with_reject=True) 

,addr_key,country,postcode,city,street,housenumber,osm_addr_in,place_id,lat,lon,...,addr_out_street,addr_out_city,addr_out_number,addr_out_country,addr_out_postcode,addr_out_other,retry_on_26,method,extra_house_nbr,rejected
0,2.164.397.226,Belgique,1330,Rixensart,Rue Robert Boisacq,9 A,"Rue Robert Boisacq, 9 A, 1330 Rixensart, Belgique",564275.0,50.712641,4.530106,...,Rue Robert Boisacq,Rixensart,None,België / Belgique / Belgien,1330,Fond du Patch,None,orig,9 A,None
1,2.279.599.374,Belgique,8908,Ieper,Streuvelswijk,18,"Streuvelswijk, 18, 8908 Ieper, Belgique",785053.0,50.859702,2.828440,...,Stijn Streuvelswijk,Ieper,None,België / Belgique / Belgien,8908,,None,orig,18,None
2,0669.833.203,Belgique,2100,Antwerpen,Boterlaarbaan,239,"Boterlaarbaan, 239, 2100 Antwerpen, Belgique",2010239.0,51.209175,4.470072,...,Boterlaarbaan,Deurne,239,België / Belgique / Belgien,2100,Eksterlaar,None,orig,239,None
3,2.243.108.073,Belgique,4460,Grâce-Hollogne,Rue de l'Aéropostale,8,"Rue de l'Aéropostale, 8, 4460 Grâce-Hollogne, ...",1445516.0,50.658359,5.483832,...,Rue de l'Aéropostale,Grâce-Hollogne,None,België / Belgique / Belgien,4460,Liège logistics,None,orig,8,None
4,2.045.477.107,Belgique,8210,Zedelgem,Koningin Astridstraat,109,"Koningin Astridstraat, 109, 8210 Zedelgem, Bel...",1763661.0,51.100407,3.167726,...,Koningin Astridstraat,Zedelgem,None,België / Belgique / Belgien,8210,,None,orig,109,"[{'index': 5.0, 'osm_addr_in': 'Koningin Astri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,0857.952.627,Belgique,8501,Kortrijk,Heulsestraat(Bis),3,"Heulsestraat, 3, 8501 Kortrijk, Belgique",2738948.0,50.834163,3.231044,...,Heulsestraat,Kortrijk,None,België / Belgique / Belgien,8501,,None,regex[init],3,None
93,2.249.583.121,Belgique,2018,Antwerpen,Desguinlei,90,"Desguinlei, 90, 2018 Antwerpen, Belgique",2323301.0,51.194730,4.407469,...,Desguinlei,Antwerpen,88-90,België / Belgique / Belgien,2018,Markgrave,None,orig,90,None
94,0834.315.707,Belgique,2018,Antwerpen,Diercxsensstraat,39,"Diercxsensstraat, 39, 2018 Antwerpen, Belgique",2338350.0,51.204519,4.398727,...,Diercxsensstraat,Antwerpen,39,België / Belgique / Belgien,2018,Brederode,None,orig,39,None
95,0857.635.002,Belgique,1700,Dilbeek,Ninoofsesteenweg,227,"Ninoofsesteenweg, 227, 1700 Dilbeek, Belgique",943444.0,50.843955,4.264415,...,Ninoofsesteenweg,Dilbeek,None,België / Belgique / Belgien,1700,,None,orig,227,"[{'index': 120.0, 'osm_addr_in': 'Ninoofsestee..."


### Batch blocs

In [ ]:
def call_ws_batch_chunks(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False, chunk_size=100): 
    ## TODO : find a better way with dask? It seems that map_partitions does not support function returning dataframes. 

    chunks = np.array_split(addr_data, addr_data.shape[0]//chunk_size)

    res= [call_ws_batch(chunk, mode=mode, 
                        check_result=check_result, 
                        structured_osm=structured_osm) for chunk in tqdm(chunks)]
    df_res = pd.concat(res, sort=False)
    return df_res

In [ ]:
df_res = call_ws_batch_chunks(addresses, chunk_size=10)
df_res

In [24]:
df_res.method.value_counts()

orig                             73
regex[init]                      19
libpostal+regex[lpost]+photon     3
nonum                             2
nostreet                          2
Name: method, dtype: int64

## Comparing options

In [128]:
addresses = get_addresses("address.csv.gz")
addresses = addresses[addresses[country_field] == "Belgique"]
addresses = addresses.sample(10000).copy()

In [129]:
results = {}
it_per_seconds=pd.DataFrame()

for check_label in ["check", "nocheck"]:
    for struct_label in ["struct", "unstruct" ]:
        print(check_label, struct_label)
        start=datetime.now()
        
        results[(check_label, struct_label)] = call_ws_batch_chunks(addresses, 
                                                                    mode="short", 
                                                                    check_result   =  check_label == "check", 
                                                                    structured_osm =  struct_label == "struct")
        
        it_per_seconds.loc[check_label, struct_label] = addresses.shape[0] / (datetime.now()-start).total_seconds()
print("Iterations per seconds:")
it_per_seconds

check struct


  0%|          | 0/100 [00:00<?, ?it/s]

check unstruct


  0%|          | 0/100 [00:00<?, ?it/s]

nocheck struct


  0%|          | 0/100 [00:00<?, ?it/s]

nocheck unstruct


  0%|          | 0/100 [00:00<?, ?it/s]

Iterations per seconds:


,struct,unstruct
check,13.331794,12.287435
nocheck,17.268480,14.389238


In [130]:
print("Match rate")
pd.DataFrame({k1: {k2: results[(k1,k2)].shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

Match rate


,check,nocheck
struct,0.9992,0.9996
unstruct,0.9997,0.9997


In [131]:
print("Match rate (without nostreet)")
pd.DataFrame({k1: {k2: results[(k1,k2)].query("method!='nostreet'").shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

Match rate (without nostreet)


,check,nocheck
struct,0.9874,0.9882
unstruct,0.9867,0.9880


In [132]:
print("Unmatched addresses")
for k1 in results:
    print(k1)
    nomatch=addresses[~addresses[addr_key_field].isin(results[k1]["addr_key"])]
    display(nomatch)
    print(nomatch[country_field].value_counts())

Unmatched addresses
('check', 'struct')


,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
2303305,2.170.213.464,Belgique,6031,Charleroi,Parc Nelson Mandela,-
876355,0679.281.397,Belgique,8420,De Haan,Vijfwegenstraat,10
371689,0472.329.028,Belgique,6662,Houffalize,Wandebourcy,5
2600140,2.223.044.812,Belgique,1831,Machelen (Brab.),Lambroekstraat,5A
639556,0627.736.587,Belgique,2310,Rijkevorsel,Langedreef,4
2127111,2.133.101.363,Belgique,9150,Kruibeke,Scheepsbouwersstraat,4
2914650,2.265.721.941,Belgique,9220,Hamme (Vl.),Driegotenkouter,106
2860,0211.355.872,Belgique,7120,Estinnes,Place(VSC),5


Belgique    8
Name: CountryFR, dtype: int64
('check', 'unstruct')


,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
2600140,2.223.044.812,Belgique,1831,Machelen (Brab.),Lambroekstraat,5A
1284523,0830.663.854,Belgique,9220,Hamme (Vl.),St. Jansstraat,27
2914650,2.265.721.941,Belgique,9220,Hamme (Vl.),Driegotenkouter,106


Belgique    3
Name: CountryFR, dtype: int64
('nocheck', 'struct')


,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
2778756,2.248.443.865,Belgique,9220,Hamme (Vl.),Schaubeke,62
371689,0472.329.028,Belgique,6662,Houffalize,Wandebourcy,5
2600140,2.223.044.812,Belgique,1831,Machelen (Brab.),Lambroekstraat,5A
2914650,2.265.721.941,Belgique,9220,Hamme (Vl.),Driegotenkouter,106


Belgique    4
Name: CountryFR, dtype: int64
('nocheck', 'unstruct')


,EntityNumber,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
2600140,2.223.044.812,Belgique,1831,Machelen (Brab.),Lambroekstraat,5A
1284523,0830.663.854,Belgique,9220,Hamme (Vl.),St. Jansstraat,27
2914650,2.265.721.941,Belgique,9220,Hamme (Vl.),Driegotenkouter,106


Belgique    3
Name: CountryFR, dtype: int64


In [133]:
vc_values = pd.DataFrame(columns=results.keys(), index=results.keys())

for k1 in results:
    vc_values.loc[k1, k1] = results[k1].shape[0]
    for k2 in results:
        if k1>k2:
            r1=results[k1]
            r2=results[k2]
            mg = r1[["addr_key", "place_id"]].merge(r2[["addr_key", "place_id"]], on="addr_key", how="outer", indicator=True)
 
            vc = mg._merge.value_counts()

            mismatches = mg[mg.place_id_x != mg.place_id_y][["addr_key"]]
            mismatches = mismatches.merge(addresses.rename({addr_key_field:"addr_key"}, axis=1))
            mismatches = mismatches.merge(r1[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches = mismatches.merge(r2[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches.columns = pd.MultiIndex.from_arrays([["Input"]*6 + [f"x:{k1}"]*5 + [f"y:{k2}"]*5, mismatches.columns])

            mismatch_values = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).fillna("") != 
                                          mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).fillna("")).any(axis=1)]
            
            mismatch_values_no_nmbr = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).drop("addr_out_number", axis=1).fillna("") != 
                                                  mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).drop("addr_out_number", axis=1).fillna("")).any(axis=1)]
            
            
            vc_label = f"{vc['both']} ({mismatches.shape[0]} - {mismatch_values.shape[0]} - {mismatch_values_no_nmbr.shape[0]}) / {vc['left_only']} / {vc['right_only']}"
            vc_values.loc[k1, k2]=vc_label

                
            print(f"{k1} vs {k2}")
            print(vc_label)
            print("-----------------------------")
            
            print(f"Only in {k1}")
            display(r1[r1.addr_key.isin(mg[mg._merge=="left_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print(f"Only in {k2}")
            display(r2[r2.addr_key.isin(mg[mg._merge=="right_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print("Mismatch on place_id")
            display(mismatches)
            
            print("Mismatch on values")
            
            display(mismatch_values)
            
            print("Mismatch on values (no nbr)")
            display(mismatch_values_no_nmbr)
            
            print("#######################")
            
# display(vc_values.fillna(""))

('check', 'unstruct') vs ('check', 'struct')
9991 (352 - 310 - 239) / 6 / 1
-----------------------------
Only in ('check', 'unstruct')


,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
0,2.170.213.464,50.407989,4.371269,22.0,orig,1116413.0,None,None,-,6031,Monceau-sur-Sambre,België / Belgique / Belgien,Belgique,6031,Charleroi,Parc Nelson Mandela,-
1,0679.281.397,51.271467,3.037741,21.0,nostreet,4068584.0,None,None,10,8420,Vlissegem,België / Belgique / Belgien,Belgique,8420,De Haan,Vijfwegenstraat,10
2,0472.329.028,50.092587,5.832763,26.0,orig,1339374.0,Wandebourcy,None,5,None,Boeur,België / Belgique / Belgien,Belgique,6662,Houffalize,Wandebourcy,5
3,0627.736.587,51.346506,4.762338,21.0,nostreet,4067757.0,None,None,4,2310,Rijkevorsel,België / Belgique / Belgien,Belgique,2310,Rijkevorsel,Langedreef,4
4,2.133.101.363,51.161173,4.301394,21.0,nostreet,4068702.0,None,None,4,9150,Kruibeke,België / Belgique / Belgien,Belgique,9150,Kruibeke,Scheepsbouwersstraat,4
5,0211.355.872,50.378813,4.090667,21.0,nostreet,4068443.0,None,None,5,7120,La Louvière,België / Belgique / Belgien,Belgique,7120,Estinnes,Place(VSC),5


Only in ('check', 'struct')


,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,extra_house_nbr,addr_out_postcode,addr_out_city,addr_out_country,CountryFR,Zipcode,MunicipalityFR,StreetFR,HouseNumber
0,0830.663.854,51.09708,4.131536,26.0,libpostal+regex[lpost]+photon,2265189.0,Sint Janstraat,None,27,9220,Hamme,België / Belgique / Belgien,Belgique,9220,Hamme (Vl.),St. Jansstraat,27


Mismatch on place_id


...

Mismatch on values


Input                                      \
          addr_key CountryFR Zipcode    MunicipalityFR   
0    2.170.213.464  Belgique    6031         Charleroi   
1     0696.696.956  Belgique    1420   Braine-l'Alleud   
2    2.224.983.822  Belgique    7050           Jurbise   
3     0474.248.638  Belgique    5310           Eghezée   
4     0608.050.933  Belgique    4845            Jalhay   
..             ...       ...     ...               ...   
369  2.139.943.724  Belgique    3500           Hasselt   
370   0827.604.691  Belgique    1325  Chaumont-Gistoux   
371  2.178.046.413  Belgique    7711          Mouscron   
372  2.031.884.437  Belgique    9320             Aalst   
373   0617.047.385  Belgique    5030          Gembloux   

                                              x:('nocheck', 'unstruct')  \
                         StreetFR HouseNumber         addr_out_street_x   
0             Parc Nelson Mandela           -                      None   
1                   Rue Hippolyte          40                      None   
2             Rue des Bruyères(J)           4        Rue Bruyères Dincq   
3        Rue de la Tombale,Aische          29         Rue de la Tombale   
4                     Tiège(SART)         101                     Tiège   
..                            ...         ...                       ...   
369                       253 STW         NaN                      None   
370  Chemin Bernard Croix, Bonlez         37b      Chemin Bernard Croix   
371                Vert Chemin(D)         112       Chemin de Bléharies   
372       Dorpsstraat(afgeschaft)        52 A               Dorpsstraat   
373     Rue de la Station, Beuzet          18         Rue de la Station   

                                                             \
    addr_out_number_x extra_house_nbr_x addr_out_postcode_x   
0                None                 -                6031   
1                None                40                1420   
2                None                 4                7050   
3                  29                29                5310   
4                None               101                4845   
..                ...               ...                 ...   
369              None               253                3500   
370               37B               37b                1325   
371              None               112                7623   
372                52              52 A                9420   
373                18                18                5030   

                        y:('nocheck', 'struct')                    \
        addr_out_city_x       addr_out_street_y addr_out_number_y   
0    Monceau-sur-Sambre   Autoroute de Wallonie              None   
1       Braine-l'Alleud        Place de la Gare              None   
2               Jurbise    Impasse des Bruyères              None   
3      Aische-En-Refail                    E411              None   
4                 Tiège                   Tiège               101   
..                  ...                     ...               ...   
369             Hasselt           Stationsplein              None   
370    Chaumont-Gistoux              Bas-Bonlez              None   
371               Rongy      Rue du chemin vert              None   
372                Erpe                     E40              None   
373            Gembloux         Route de Beuzet                18   

                                                             
    extra_house_nbr_y addr_out_postcode_y   addr_out_city_y  
0                   -                6041         Gosselies  
1                  40                1420   Braine-l'Alleud  
2                   4                7050              Mons  
3                  29                5310     Saint-Germain  
4                 101                4845             Tiège  
..                ...                 ...               ...  
369               253                3500           Hasselt  
370   

Mismatch on values (no nbr)


Input                                         \
          addr_key CountryFR Zipcode       MunicipalityFR   
0    2.170.213.464  Belgique    6031            Charleroi   
1     0696.696.956  Belgique    1420      Braine-l'Alleud   
2    2.224.983.822  Belgique    7050              Jurbise   
3     0474.248.638  Belgique    5310              Eghezée   
5    2.150.463.175  Belgique    9520  Sint-Lievens-Houtem   
..             ...       ...     ...                  ...   
369  2.139.943.724  Belgique    3500              Hasselt   
370   0827.604.691  Belgique    1325     Chaumont-Gistoux   
371  2.178.046.413  Belgique    7711             Mouscron   
372  2.031.884.437  Belgique    9320                Aalst   
373   0617.047.385  Belgique    5030             Gembloux   

                                              x:('nocheck', 'unstruct')  \
                         StreetFR HouseNumber         addr_out_street_x   
0             Parc Nelson Mandela           -                      None   
1                   Rue Hippolyte          40                      None   
2             Rue des Bruyères(J)           4        Rue Bruyères Dincq   
3        Rue de la Tombale,Aische          29         Rue de la Tombale   
5                  Gentsesteenweg          1.                      None   
..                            ...         ...                       ...   
369                       253 STW         NaN                      None   
370  Chemin Bernard Croix, Bonlez         37b      Chemin Bernard Croix   
371                Vert Chemin(D)         112       Chemin de Bléharies   
372       Dorpsstraat(afgeschaft)        52 A               Dorpsstraat   
373     Rue de la Station, Beuzet          18         Rue de la Station   

                                                             \
    addr_out_number_x extra_house_nbr_x addr_out_postcode_x   
0                None                 -                6031   
1                None                40                1420   
2                None                 4                7050   
3                  29                29                5310   
5                None                1.                9520   
..                ...               ...                 ...   
369              None               253                3500   
370               37B               37b                1325   
371              None               112                7623   
372                52              52 A                9420   
373                18                18                5030   

                         y:('nocheck', 'struct')                    \
         addr_out_city_x       addr_out_street_y addr_out_number_y   
0     Monceau-sur-Sambre   Autoroute de Wallonie              None   
1        Braine-l'Alleud        Place de la Gare              None   
2                Jurbise    Impasse des Bruyères              None   
3       Aische-En-Refail                    E411              None   
5    Sint-Lievens-Houtem              Marktplein                 3   
..                   ...                     ...               ...   
369              Hasselt           Stationsplein              None   
370     Chaumont-Gistoux              Bas-Bonlez              None   
371                Rongy      Rue du chemin vert              None   
372                 Erpe                     E40              None   
373             Gembloux         Route de Beuzet                18   

                                                                
    extra_house_nbr_y addr_out_postcode_y      addr_out_city_y  
0                   -                6041            Gosselies  
1                  40                1420      Braine-l'Alleud  
2                   4                7050                 Mons  
3                  29                5310        Saint-Germain  
5                  1.                9520  Sint-Lievens-Houtem  
..                ...                 ...                

#######################


In [134]:
print("Common in both (disagree on place_id - disagree on values - disagree on values, ignoring number) / results only for row / results only for columns")
vc_values.fillna("")

Common in both (disagree on place_id - disagree on values - disagree on values, ignoring number) / results only for row / results only for columns


check  \
                                          struct   
check   struct                              9992   
        unstruct  9991 (352 - 310 - 239) / 6 / 1   
nocheck struct    9991 (271 - 271 - 264) / 5 / 1   
        unstruct  9991 (543 - 503 - 426) / 6 / 1   

                                                  \
                                        unstruct   
check   struct                                     
        unstruct                            9997   
nocheck struct    9995 (567 - 525 - 448) / 1 / 2   
        unstruct  9997 (213 - 213 - 207) / 0 / 0   

                                         nocheck           
                                          struct unstruct  
check   struct                                             
        unstruct                                           
nocheck struct                              9996           
        unstruct  9995 (374 - 334 - 262) / 2 / 1     9997